In [1]:
import kagglehub
from tqdm import tqdm
from pathlib import Path
import pandas as pd
from transformers import pipeline
from datasets import Dataset, load_dataset
from transformers import set_seed
set_seed(42)

# Load the original sentiment analysis dataset

In [2]:
# Download latest version
path = kagglehub.dataset_download("kashishparmar02/social-media-sentiments-analysis-dataset")

print("Path to dataset files:", path)

100%|██████████| 50.9k/50.9k [00:00<00:00, 12.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/kashishparmar02/social-media-sentiments-analysis-dataset/versions/3


In [3]:
df = pd.read_csv(Path(path) / "sentimentdataset.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [4]:
df = df.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
df.head()

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


# Load the translated dataframe with Gemini-2.5 Pro

In [26]:
gemini_df = pd.read_csv("/notebooks/translations-gemini-2.5pro.csv")
gemini_df["English"] = gemini_df.apply(lambda x: x["English"].strip(), axis=1)

In [27]:
gemini_df

,English,Myanmar
0,Enjoying a beautiful day at the park!,ပန်းခြံထဲမှာ လှပတဲ့နေ့လေးတစ်နေ့ကို ပျော်ရွှင်စ...
1,Traffic was terrible this morning.,ဒီမနက် လမ်းအရမ်းပိတ်တယ်။
2,Just finished an amazing workout! 💪,အရမ်းကောင်းတဲ့ လေ့ကျင့်ခန်းတစ်ခု ခုပဲပြီးသွားပ...
3,Excited about the upcoming weekend getaway!,လာမယ့် စနေ၊ တနင်္ဂနွေ အပန်းဖြေခရီးအတွက် စိတ်လှ...
4,Trying out a new recipe for dinner tonight.,ဒီည ညစာအတွက် ဟင်းချက်နည်းအသစ်တစ်ခု စမ်းသပ်ချက်...
...,...,...
727,Collaborating on a science project that receiv...,ဒေသဆိုင်ရာ ပြပွဲတစ်ခုမှာ အသိအမှတ်ပြုမှု ရရှိခဲ...
728,Attending a surprise birthday party organized ...,သူငယ်ချင်းတွေက စီစဉ်ပေးတဲ့ အံ့အားသင့်ဖွယ် မွေး...
729,Successfully fundraising for a school charity ...,ကျောင်း ပရဟိတ အစီအစဉ်တစ်ခုအတွက် အောင်မြင်စွာ ရ...
730,"Participating in a multicultural festival, cel...",ယဉ်ကျေးမှုပေါင်းစုံ ပွဲတော်တစ်ခုမှာ ပါဝင်နေပါတ...


# Map the translated Myanmar Text into Original DataFrame

In [ ]:
def map_eng_mm(row):
    myanmar_text = gemini_df[gemini_df["English"] == row["Text"].strip()]['Myanmar']
    myanmar_text = myanmar_text.drop_duplicates()
    
    return myanmar_text.item()
    
df["Text-MM"] = df.apply(map_eng_mm, axis=1)

In [47]:
df.head()

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,Text-MM
0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12,ပန်းခြံထဲမှာ လှပတဲ့နေ့လေးတစ်နေ့ကို ပျော်ရွှင်စ...
1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8,ဒီမနက် လမ်းအရမ်းပိတ်တယ်။
2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15,အရမ်းကောင်းတဲ့ လေ့ကျင့်ခန်းတစ်ခု ခုပဲပြီးသွားပ...
3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18,လာမယ့် စနေ၊ တနင်္ဂနွေ အပန်းဖြေခရီးအတွက် စိတ်လှ...
4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19,ဒီည ညစာအတွက် ဟင်းချက်နည်းအသစ်တစ်ခု စမ်းသပ်ချက်...


In [48]:
ordered_columns = ["Text", "Text-MM", "Sentiment", "Timestamp", "User", 
                  "Platform", "Hashtags", "Retweets", "Likes", 
                  "Country", "Year", "Month", "Day", "Hour"]

# Filter to only include columns that actually exist
ordered_columns = [col for col in ordered_columns if col in df.columns]
df = df[ordered_columns]

In [49]:
df.head()

,Text,Text-MM,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park! ...,ပန်းခြံထဲမှာ လှပတဲ့နေ့လေးတစ်နေ့ကို ပျော်ရွှင်စ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,Traffic was terrible this morning. ...,ဒီမနက် လမ်းအရမ်းပိတ်တယ်။,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,Just finished an amazing workout! 💪 ...,အရမ်းကောင်းတဲ့ လေ့ကျင့်ခန်းတစ်ခု ခုပဲပြီးသွားပ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway! ...,လာမယ့် စနေ၊ တနင်္ဂနွေ အပန်းဖြေခရီးအတွက် စိတ်လှ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight. ...,ဒီည ညစာအတွက် ဟင်းချက်နည်းအသစ်တစ်ခု စမ်းသပ်ချက်...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


# Upload to HuggingFace

In [51]:
ds = Dataset.from_pandas(df)
ds.push_to_hub("chuuhtetnaing/myanmar-social-media-sentiment-analysis-dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/chuuhtetnaing/myanmar-social-media-sentiment-analysis-dataset/commit/9ffd58b98eab714e9100477d0975278b1b9d19e3', commit_message='Upload dataset', commit_description='', oid='9ffd58b98eab714e9100477d0975278b1b9d19e3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/chuuhtetnaing/myanmar-social-media-sentiment-analysis-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='chuuhtetnaing/myanmar-social-media-sentiment-analysis-dataset'), pr_revision=None, pr_num=None)

In [64]:
# Create the pipeline
pipe = pipeline("translation", model="facebook/nllb-200-3.3B", device=0)  # device=0 for first GPU

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [65]:
pipe("Weekend road trip to explore scenic views.", src_lang="eng_Latn", tgt_lang="mya_Mymr")

[{'translation_text': 'အပန်းဖြေစခန်းကို အလည်သွားပြီး ရှုခင်းကို လေ့လာကြည့်တယ်။'}]

In [66]:
dataset = Dataset.from_pandas(df)

# Define a function to translate in batches
def translate_batch(examples):
    translations = pipe(
        examples["Text"], 
        src_lang="eng_Latn", 
        tgt_lang="mya_Mymr",
        batch_size=8
    )
    return {
        "Text": examples["Text"],
        "Text_MM": [t["translation_text"] for t in translations],
        "Sentiment": examples["Sentiment"],
        "Timestamp": examples["Timestamp"],
        "User": examples["User"],
        "Platform": examples["Platform"],
        "Hashtags": examples["Hashtags"],
        "Retweets": examples["Retweets"],
        "Likes": examples["Likes"],
        "Country": examples["Country"],
        "Year": examples["Year"],
        "Month": examples["Month"],
        "Day": examples["Day"],
        "Hour": examples["Hour"]
    }

In [7]:
# Apply the translation function to the entire dataset
# This will process in batches automatically
translated_dataset = dataset.map(
    translate_batch,
    batched=True,
    batch_size=8,  # You can adjust this batch size
    remove_columns=dataset.column_names
)



Map:   0%|          | 0/732 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
ordered_columns = ["Text", "Text_MM", "Sentiment", "Timestamp", "User", 
                  "Platform", "Hashtags", "Retweets", "Likes", 
                  "Country", "Year", "Month", "Day", "Hour"]

df = translated_dataset.to_pandas()
# Filter to only include columns that actually exist
ordered_columns = [col for col in ordered_columns if col in df.columns]
df = df[ordered_columns]

In [69]:
df.iloc[-10:]

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
722,Winning a friendly sports competition against ...,Happy,2023-10-20 14:15:00,SportsVictoryHighSchool,Twitter,#SportsTriumph #HighSchoolAthletics,26.0,50.0,USA,2023,10,20,14
723,Receiving a heartfelt letter from a pen pal in...,Happy,2023-10-21 17:30:00,PenPalConnectionHighSchool,Facebook,#GlobalFriendship #HighSchoolCorrespondence,23.0,45.0,Canada,2023,10,21,17
724,Creating a beautiful mural with fellow art ent...,Happy,2023-10-22 20:45:00,MuralCreationHighSchool,Instagram,#ArtCollaboration #HighSchoolCreativity,22.0,43.0,UK,2023,10,22,20
725,Participating in a school-wide art exhibition....,Happy,2015-11-10 16:45:00,ArtExhibitionJoyHighSchool,Instagram,#ArtisticExpression #HighSchoolCreativity,23.0,44.0,USA,2015,11,10,16
726,Achieving a personal best in a track and field...,Happy,2016-09-14 12:30:00,TrackFieldSuccessHighSchool,Twitter,#AthleticAchievement #HighSchoolSports,26.0,51.0,Canada,2016,9,14,12
727,Collaborating on a science project that receiv...,Happy,2017-08-18 18:20:00,ScienceProjectSuccessHighSchool,Facebook,#ScienceFairWinner #HighSchoolScience,20.0,39.0,UK,2017,8,18,18
728,Attending a surprise birthday party organized ...,Happy,2018-06-22 14:15:00,BirthdayPartyJoyHighSchool,Instagram,#SurpriseCelebration #HighSchoolFriendship,25.0,48.0,USA,2018,6,22,14
729,Successfully fundraising for a school charity ...,Happy,2019-04-05 17:30:00,CharityFundraisingTriumphHighSchool,Twitter,#CommunityGiving #HighSchoolPhilanthropy,22.0,42.0,Canada,2019,4,5,17
730,"Participating in a multicultural festival, cel...",Happy,2020-02-29 20:45:00,MulticulturalFestivalJoyHighSchool,Facebook,#CulturalCelebration #HighSchoolUnity,21.0,43.0,UK,2020,2,29,20
731,Organizing a virtual talent show during challe...,Happy,2020-11-15 15:15:00,VirtualTalentShowSuccessHighSchool,Instagram,#VirtualEntertainment #HighSchoolPositivity,24.0,47.0,USA,2020,11,15,15


In [74]:
dataset = Dataset.from_pandas(df.iloc[-10:])

# Define a function to translate in batches
def translate(example):
    translation = pipe(
        example["Text"], 
        src_lang="eng_Latn", 
        tgt_lang="mya_Mymr"
    )[0]["translation_text"]
    
    print(example["Text"])
    print(translation)
    print()
    return {"Text_MM": translation}

In [83]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [93]:
def translate(text, src_lang="eng_Latn", tgt_lang="mya_Mymr"):
    # Set the language tokens
    tokenizer.src_lang = src_lang
    
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    
    # Get the target language token ID
    forced_bos_token_id = tokenizer.lang_code_to_id[tgt_lang]
    
    # Generate translation
    outputs = model.generate(
        **inputs,
        forced_bos_token_id=forced_bos_token_id,
        max_length=128
    )
    
    # Decode the output tokens
    translation = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    return translation

In [ ]:
translate("An amusing incident brightened up my day!")

AttributeError: NllbTokenizerFast has no attribute lang_code_to_id

In [112]:
article = "Receiving a heartfelt letter from a pen pal in another country. Connecting across the globe!".split(".")
article

['Receiving a heartfelt letter from a pen pal in another country',
 ' Connecting across the globe!']

In [113]:
article = "Receiving a heartfelt letter from a pen pal in another country. Connecting across the globe!".split(".")
inputs = tokenizer(article, return_tensors="pt")

translated_tokens = model.generate(
    **inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids("mya_Mymr"), max_length=500
)
tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [104]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Smutná událost mi dnes usmála den.'

In [114]:
pipe("Receiving a heartfelt letter from a pen pal in another country. Connecting across the globe!".split("."), src_lang="eng_Latn", tgt_lang="mya_Mymr")

[{'translation_text': 'အခြားနိုင်ငံမှ စာရေးဖော်ထံမှ နှလုံးသားကိုဖုံးလွှမ်းစေသော စာရခြင်း'},
 {'translation_text': 'ကမ္ဘာတစ်လွှားမှာ ဆက်သွယ်မှုပါ။'}]

In [75]:
# Apply the translation function to the entire dataset
# This will process in batches automatically
translated_dataset = dataset.map(translate)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Winning a friendly sports competition against rival schools. Victory celebrations in full swing! 
မိတ်ဆွေများအား အားကစားပြိုင်ပွဲတွင် ပြိုင်ဘက်ကျောင်းများအား အနိုင်ရခြင်း။

Receiving a heartfelt letter from a pen pal in another country. Connecting across the globe! 
[စာမျက်နှာ ၂၃ ပါ ရုပ်ပုံ]

Creating a beautiful mural with fellow art enthusiasts. The power of collaboration and creativity! 
အနုပညာ ဝါသနာအိုးများနဲ့အတူ လှပတဲ့ နံရံဆေးရေးပန်းချီကို ဖန်တီးခြင်း။

Participating in a school-wide art exhibition. Witnessing creativity and spreading positive vibes! 
ကျောင်းတစ်ဝှမ်းလုံး အနုပညာပြပွဲတွင် ပါဝင်ခြင်း ဖန်တီးမှုကို မျက်မြင်တွေ့ပြီး အပြုသဘော လှိုင်းများကို ပျံ့နှံ့စေခြင်း!

Achieving a personal best in a track and field competition. The thrill of victory and self-improvement! 
[စာမျက်နှာ ၂၁ ပါ ရုပ်ပုံ]

Collaborating on a science project that received recognition at a regional fair. Science triumphs and smiles! 
ဒေသတွင်း ပြပွဲတစ်ခုတွင် အသိအမှတ်ပြုခံရသော သိပ္ပံပရောဂျက်တွင် ပူးပေါင်းခြင်း

In [76]:
ordered_columns = ["Text", "Text_MM", "Sentiment", "Timestamp", "User", 
                  "Platform", "Hashtags", "Retweets", "Likes", 
                  "Country", "Year", "Month", "Day", "Hour"]

df = translated_dataset.to_pandas()
# Filter to only include columns that actually exist
ordered_columns = [col for col in ordered_columns if col in df.columns]
df = df[ordered_columns]

In [73]:
df

,Text,Text_MM,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Winning a friendly sports competition against ...,မိတ်ဆွေများအား အားကစားပြိုင်ပွဲတွင် ပြိုင်ဘက်က...,Happy,2023-10-20 14:15:00,SportsVictoryHighSchool,Twitter,#SportsTriumph #HighSchoolAthletics,26.0,50.0,USA,2023,10,20,14
1,Receiving a heartfelt letter from a pen pal in...,[စာမျက်နှာ ၂၃ ပါ ရုပ်ပုံ],Happy,2023-10-21 17:30:00,PenPalConnectionHighSchool,Facebook,#GlobalFriendship #HighSchoolCorrespondence,23.0,45.0,Canada,2023,10,21,17
2,Creating a beautiful mural with fellow art ent...,အနုပညာ ဝါသနာအိုးများနဲ့အတူ လှပတဲ့ နံရံဆေးရေးပန...,Happy,2023-10-22 20:45:00,MuralCreationHighSchool,Instagram,#ArtCollaboration #HighSchoolCreativity,22.0,43.0,UK,2023,10,22,20
3,Participating in a school-wide art exhibition....,ကျောင်းတစ်ဝှမ်းလုံး အနုပညာပြပွဲတွင် ပါဝင်ခြင်း...,Happy,2015-11-10 16:45:00,ArtExhibitionJoyHighSchool,Instagram,#ArtisticExpression #HighSchoolCreativity,23.0,44.0,USA,2015,11,10,16
4,Achieving a personal best in a track and field...,[စာမျက်နှာ ၂၁ ပါ ရုပ်ပုံ],Happy,2016-09-14 12:30:00,TrackFieldSuccessHighSchool,Twitter,#AthleticAchievement #HighSchoolSports,26.0,51.0,Canada,2016,9,14,12
5,Collaborating on a science project that receiv...,ဒေသတွင်း ပြပွဲတစ်ခုတွင် အသိအမှတ်ပြုခံရသော သိပ္...,Happy,2017-08-18 18:20:00,ScienceProjectSuccessHighSchool,Facebook,#ScienceFairWinner #HighSchoolScience,20.0,39.0,UK,2017,8,18,18
6,Attending a surprise birthday party organized ...,မိတ်ဆွေတွေက ကျင်းပတဲ့ မမျှော်လင့်တဲ့ မွေးနေ့ပါ...,Happy,2018-06-22 14:15:00,BirthdayPartyJoyHighSchool,Instagram,#SurpriseCelebration #HighSchoolFriendship,25.0,48.0,USA,2018,6,22,14
7,Successfully fundraising for a school charity ...,ကျောင်းပရဟိတလုပ်ငန်းတစ်ခုအတွက် ငွေကြေးထောက်ပံ့...,Happy,2019-04-05 17:30:00,CharityFundraisingTriumphHighSchool,Twitter,#CommunityGiving #HighSchoolPhilanthropy,22.0,42.0,Canada,2019,4,5,17
8,"Participating in a multicultural festival, cel...",ယဉ်ကျေးမှုစုံလင်သောပွဲတော်တွင် ပါဝင်ခြင်း၊ ဂီတ...,Happy,2020-02-29 20:45:00,MulticulturalFestivalJoyHighSchool,Facebook,#CulturalCelebration #HighSchoolUnity,21.0,43.0,UK,2020,2,29,20
9,Organizing a virtual talent show during challe...,စိန်ခေါ်မှုရှိတဲ့ အချိန်တွေမှာ ပင်ကိုယ်စွမ်းရည...,Happy,2020-11-15 15:15:00,VirtualTalentShowSuccessHighSchool,Instagram,#VirtualEntertainment #HighSchoolPositivity,24.0,47.0,USA,2020,11,15,15


In [51]:
previous_ds = load_dataset("chuuhtetnaing/myanmar-social-media-sentiment-analysis-dataset")
previous_df = previous_ds['train'].to_pandas()

In [52]:
previous_df = previous_df.rename(columns={"Text_MM": "Text_MM_old"})

In [53]:
previous_df

,Text,Text_MM_old,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park! ...,ပန်းခြံထဲမှာ လှပတဲ့ နေ့ကို ခံစားရင်းပေါ့။,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,Traffic was terrible this morning. ...,ဒီမနက်က ယာဉ်ကြောက အရမ်းကို ဆိုးတယ်။,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,Just finished an amazing workout! 💪 ...,အံ့အားသင့်စရာ လေ့ကျင့်ခန်း ပြီးသွားပြီ။,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway! ...,လာမယ့် သီတင်းပတ်ရဲ့ ထွက်ခွာမှုအတွက် စိတ်လှုပ်ရ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight. ...,ဒီညစာအတွက် ဟင်းချက်နည်း အသစ်တစ်ခု စမ်းနေတယ်။,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,ဒေသတွင်း ပြပွဲတစ်ခုတွင် အသိအမှတ်ပြုခံရသော သိပ္...,Happy,2017-08-18 18:20:00,ScienceProjectSuccessHighSchool,Facebook,#ScienceFairWinner #HighSchoolScience,20.0,39.0,UK,2017,8,18,18
728,Attending a surprise birthday party organized ...,မိတ်ဆွေတွေက ကျင်းပတဲ့ မမျှော်လင့်တဲ့ မွေးနေ့ပါ...,Happy,2018-06-22 14:15:00,BirthdayPartyJoyHighSchool,Instagram,#SurpriseCelebration #HighSchoolFriendship,25.0,48.0,USA,2018,6,22,14
729,Successfully fundraising for a school charity ...,ကျောင်းပရဟိတလုပ်ငန်းတစ်ခုအတွက် ငွေကြေးထောက်ပံ့...,Happy,2019-04-05 17:30:00,CharityFundraisingTriumphHighSchool,Twitter,#CommunityGiving #HighSchoolPhilanthropy,22.0,42.0,Canada,2019,4,5,17
730,"Participating in a multicultural festival, cel...",ယဉ်ကျေးမှုစုံလင်သောပွဲတော်တွင် ပါဝင်ခြင်း၊ ဂီတ...,Happy,2020-02-29 20:45:00,MulticulturalFestivalJoyHighSchool,Facebook,#CulturalCelebration #HighSchoolUnity,21.0,43.0,UK,2020,2,29,20


In [59]:
translated_dataset[-1]

{'Text': "Organizing a virtual talent show during challenging times, bringing smiles to classmates' faces! ",
 'Sentiment': ' Happy ',
 'Timestamp': '2020-11-15 15:15:00',
 'User': ' VirtualTalentShowSuccessHighSchool ',
 'Platform': ' Instagram ',
 'Hashtags': ' #VirtualEntertainment #HighSchoolPositivity ',
 'Retweets': 24.0,
 'Likes': 47.0,
 'Country': ' USA',
 'Year': 2020,
 'Month': 11,
 'Day': 15,
 'Hour': 15,
 'Text_MM': 'စိန်ခေါ်မှုရှိတဲ့ အချိန်တွေမှာ ပင်ကိုယ်စွမ်းရည် ပြပွဲတစ်ခုကို ဖန်တီးခြင်းပါ။ အတန်းဖော်တွေရဲ့ မျက်နှာတွေပေါ် အပြုံးတွေတင်ရင်းပေါ့။'}

In [54]:
df['Text_MM_old'] = previous_df['Text_MM_old']

In [55]:
df

,Text,Text_MM,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,Text_MM_old
0,Enjoying a beautiful day at the park! ...,ပန်းခြံထဲမှာ လှပတဲ့ နေ့ကို ခံစားရင်းပေါ့။,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12,ပန်းခြံထဲမှာ လှပတဲ့ နေ့ကို ခံစားရင်းပေါ့။
1,Traffic was terrible this morning. ...,ဒီမနက်က ယာဉ်ကြောက အရမ်းကို ဆိုးတယ်။,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8,ဒီမနက်က ယာဉ်ကြောက အရမ်းကို ဆိုးတယ်။
2,Just finished an amazing workout! 💪 ...,အံ့အားသင့်စရာ လေ့ကျင့်ခန်း ပြီးသွားပြီ။,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15,အံ့အားသင့်စရာ လေ့ကျင့်ခန်း ပြီးသွားပြီ။
3,Excited about the upcoming weekend getaway! ...,လာမယ့် သီတင်းပတ်ရဲ့ ထွက်ခွာမှုအတွက် စိတ်လှုပ်ရ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18,လာမယ့် သီတင်းပတ်ရဲ့ ထွက်ခွာမှုအတွက် စိတ်လှုပ်ရ...
4,Trying out a new recipe for dinner tonight. ...,ဒီညစာအတွက် ဟင်းချက်နည်း အသစ်တစ်ခု စမ်းနေတယ်။,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19,ဒီညစာအတွက် ဟင်းချက်နည်း အသစ်တစ်ခု စမ်းနေတယ်။
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,ဒေသတွင်း ပြပွဲတစ်ခုတွင် အသိအမှတ်ပြုခံရသော သိပ္...,Happy,2017-08-18 18:20:00,ScienceProjectSuccessHighSchool,Facebook,#ScienceFairWinner #HighSchoolScience,20.0,39.0,UK,2017,8,18,18,ဒေသတွင်း ပြပွဲတစ်ခုတွင် အသိအမှတ်ပြုခံရသော သိပ္...
728,Attending a surprise birthday party organized ...,မိတ်ဆွေတွေက ကျင်းပတဲ့ မမျှော်လင့်တဲ့ မွေးနေ့ပါ...,Happy,2018-06-22 14:15:00,BirthdayPartyJoyHighSchool,Instagram,#SurpriseCelebration #HighSchoolFriendship,25.0,48.0,USA,2018,6,22,14,မိတ်ဆွေတွေက ကျင်းပတဲ့ မမျှော်လင့်တဲ့ မွေးနေ့ပါ...
729,Successfully fundraising for a school charity ...,ကျောင်းပရဟိတလုပ်ငန်းတစ်ခုအတွက် ငွေကြေးထောက်ပံ့...,Happy,2019-04-05 17:30:00,CharityFundraisingTriumphHighSchool,Twitter,#CommunityGiving #HighSchoolPhilanthropy,22.0,42.0,Canada,2019,4,5,17,ကျောင်းပရဟိတလုပ်ငန်းတစ်ခုအတွက် ငွေကြေးထောက်ပံ့...
730,"Participating in a multicultural festival, cel...",ယဉ်ကျေးမှုစုံလင်သောပွဲတော်တွင် ပါဝင်ခြင်း၊ ဂီတ...,Happy,2020-02-29 20:45:00,MulticulturalFestivalJoyHighSchool,Facebook,#CulturalCelebration #HighSchoolUnity,21.0,43.0,UK,2020,2,29,20,ယဉ်ကျေးမှုစုံလင်သောပွဲတော်တွင် ပါဝင်ခြင်း၊ ဂီတ...


In [56]:
ordered_columns = ["Text", "Text_MM", "Text_MM_old", "Sentiment", "Timestamp", "User", 
                  "Platform", "Hashtags", "Retweets", "Likes", 
                  "Country", "Year", "Month", "Day", "Hour"]

# Filter to only include columns that actually exist
# ordered_columns = [col for col in ordered_columns if col in df.columns]
df = df[ordered_columns]

In [57]:
df

,Text,Text_MM,Text_MM_old,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park! ...,ပန်းခြံထဲမှာ လှပတဲ့ နေ့ကို ခံစားရင်းပေါ့။,ပန်းခြံထဲမှာ လှပတဲ့ နေ့ကို ခံစားရင်းပေါ့။,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,Traffic was terrible this morning. ...,ဒီမနက်က ယာဉ်ကြောက အရမ်းကို ဆိုးတယ်။,ဒီမနက်က ယာဉ်ကြောက အရမ်းကို ဆိုးတယ်။,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,Just finished an amazing workout! 💪 ...,အံ့အားသင့်စရာ လေ့ကျင့်ခန်း ပြီးသွားပြီ။,အံ့အားသင့်စရာ လေ့ကျင့်ခန်း ပြီးသွားပြီ။,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway! ...,လာမယ့် သီတင်းပတ်ရဲ့ ထွက်ခွာမှုအတွက် စိတ်လှုပ်ရ...,လာမယ့် သီတင်းပတ်ရဲ့ ထွက်ခွာမှုအတွက် စိတ်လှုပ်ရ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight. ...,ဒီညစာအတွက် ဟင်းချက်နည်း အသစ်တစ်ခု စမ်းနေတယ်။,ဒီညစာအတွက် ဟင်းချက်နည်း အသစ်တစ်ခု စမ်းနေတယ်။,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Collaborating on a science project that receiv...,ဒေသတွင်း ပြပွဲတစ်ခုတွင် အသိအမှတ်ပြုခံရသော သိပ္...,ဒေသတွင်း ပြပွဲတစ်ခုတွင် အသိအမှတ်ပြုခံရသော သိပ္...,Happy,2017-08-18 18:20:00,ScienceProjectSuccessHighSchool,Facebook,#ScienceFairWinner #HighSchoolScience,20.0,39.0,UK,2017,8,18,18
728,Attending a surprise birthday party organized ...,မိတ်ဆွေတွေက ကျင်းပတဲ့ မမျှော်လင့်တဲ့ မွေးနေ့ပါ...,မိတ်ဆွေတွေက ကျင်းပတဲ့ မမျှော်လင့်တဲ့ မွေးနေ့ပါ...,Happy,2018-06-22 14:15:00,BirthdayPartyJoyHighSchool,Instagram,#SurpriseCelebration #HighSchoolFriendship,25.0,48.0,USA,2018,6,22,14
729,Successfully fundraising for a school charity ...,ကျောင်းပရဟိတလုပ်ငန်းတစ်ခုအတွက် ငွေကြေးထောက်ပံ့...,ကျောင်းပရဟိတလုပ်ငန်းတစ်ခုအတွက် ငွေကြေးထောက်ပံ့...,Happy,2019-04-05 17:30:00,CharityFundraisingTriumphHighSchool,Twitter,#CommunityGiving #HighSchoolPhilanthropy,22.0,42.0,Canada,2019,4,5,17
730,"Participating in a multicultural festival, cel...",ယဉ်ကျေးမှုစုံလင်သောပွဲတော်တွင် ပါဝင်ခြင်း၊ ဂီတ...,ယဉ်ကျေးမှုစုံလင်သောပွဲတော်တွင် ပါဝင်ခြင်း၊ ဂီတ...,Happy,2020-02-29 20:45:00,MulticulturalFestivalJoyHighSchool,Facebook,#CulturalCelebration #HighSchoolUnity,21.0,43.0,UK,2020,2,29,20


In [58]:
df.to_csv("to_check_manually.csv", index=False)

In [29]:
reordered_dataset = Dataset.from_pandas(df)

In [30]:
reordered_dataset

Dataset({
    features: ['Text', 'Text_MM', 'Sentiment', 'Timestamp', 'User', 'Platform', 'Hashtags', 'Retweets', 'Likes', 'Country', 'Year', 'Month', 'Day', 'Hour'],
    num_rows: 732
})

In [31]:
reordered_dataset.push_to_hub("chuuhtetnaing/myanmar-social-media-sentiment-analysis-dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/chuuhtetnaing/myanmar-social-media-sentiment-analysis-dataset/commit/6c00516d603b1822b4c1fcc7fd20e042b727e645', commit_message='Upload dataset', commit_description='', oid='6c00516d603b1822b4c1fcc7fd20e042b727e645', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/chuuhtetnaing/myanmar-social-media-sentiment-analysis-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='chuuhtetnaing/myanmar-social-media-sentiment-analysis-dataset'), pr_revision=None, pr_num=None)